In [1]:
import pandas as pd

In [5]:
# create path to Json file
folder = "C:\\Users\\Giuseppe Branca\\Desktop\\yelp_dataset\\"

business_path = "yelp_academic_dataset_business.json"
checkin_path = "yelp_academic_dataset_checkin.json"
review_path = "yelp_academic_dataset_review.json"
user_path = "yelp_academic_dataset_user.json"

# read the Json file
business = pd.read_json(folder + business_path, lines=True)



In [7]:
# initial shape of business dataset

print(business.shape)

(150346, 14)


In [15]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


FIRST FILTER: GEOGRAPHIC AREA (TAMPA BAY).  
I will keep only Tampa, St. Petersburg, Oldsmar, Safety Harbor

In [211]:

# "city" contains: tampa, pete, safety, oldsmar

tampa_bay = business[business["city"].str.contains("tampa|pete|safety|oldsmar", case=False, na=False)]

In [212]:
tampa_bay.shape

(13224, 14)

SECOND FILTER: KEEPING ONLY RESTAURANTS.  
I will filter by restaurants, bars and food keywords.  
I will also eliminate airport and malls related businesses (not interested for the purpose of the project).  

In [213]:
# keeping only rows that contains restaur, bar, food in ""business categories analysis.ipynb"

tampa_bay = tampa_bay[tampa_bay["categories"].str.contains("restaur|bar|food", case=False, na=False)]

tampa_bay.shape

(5759, 14)

In [214]:
# eliminating airport, hotel, mall, shopping related 

tampa_bay = tampa_bay[~tampa_bay["categories"].str.contains("airport|hotel|mall|shopping", case=False, na=False)]

tampa_bay.shape



(5288, 14)

In [215]:
# eliminate rows with "barre" & "barber" in ""business categories analysis.ipynb"

tampa_bay = tampa_bay[~tampa_bay["categories"].str.contains("barre|barber", case=False, na=False)]

tampa_bay.shape





(5140, 14)

THIRD STEP: CLEANING THE "CATEGORIES" COLUMN.  
"categories" are inconsistent strings,  
and I would like to get a single word to represent the category.  
I will use NLP for the cleaning, removing stop words,  
counting the frequency of each word in order to get a list of categories,  
and create a new column "final_category" to obtain consistency.

In [216]:

# Define a function to count the exact word "restaurant"
def count_exact_word(word, text):
    words = text.lower().split(", ")
    return words.count(word)

# Apply the function to each row in the DataFrame and sum the counts
exact_restaurant_count = tampa_bay["categories"].apply(lambda x: count_exact_word("restaurant", x)).sum()

print(f'The word "restaurant" appears {exact_restaurant_count} times in the "categories" column.')



The word "restaurant" appears 0 times in the "categories" column.


In [217]:
# removing "restaurants", "food", "bars" from "categories" column

tampa_bay["categories"] = tampa_bay["categories"].str.replace("restaurants|food|bars", "", case=False, regex=True)


In [218]:
tampa_bay.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, , , Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
54,0qNpTGTcqPwOLi2hADx4Xw,Charlie's Market,2815 E Sligh Ave,Tampa,FL,33610,28.010360,-82.430042,3.0,9,1,"{'BusinessParking': '{'garage': False, 'street...",", Grocery, Convenience Stores",None
58,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.046203,-82.505053,4.0,23,0,"{'BusinessParking': '{'garage': False, 'street...",", American (New), Italian","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:..."
59,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.960514,-82.506127,4.0,35,0,"{'BusinessParking': '{'garage': False, 'street...",", Pizza","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
93,RK6-cJ9hj53RzOlCBmpT-g,Impasto,,Tampa,FL,33611,27.890814,-82.502346,5.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",", Italian, Trucks,","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [219]:
# removing empty spaces from "categories" column

# Removing empty spaces from "categories" column
tampa_bay["categories"] = tampa_bay["categories"].str.strip()

# Removing empty spaces between commas
tampa_bay["categories"] = tampa_bay["categories"].str.replace(r',\s*,', ',', regex=True)

# Display the first 10 rows to verify the changes
tampa_bay.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, , Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
54,0qNpTGTcqPwOLi2hADx4Xw,Charlie's Market,2815 E Sligh Ave,Tampa,FL,33610,28.010360,-82.430042,3.0,9,1,"{'BusinessParking': '{'garage': False, 'street...",", Grocery, Convenience Stores",None
58,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.046203,-82.505053,4.0,23,0,"{'BusinessParking': '{'garage': False, 'street...",", American (New), Italian","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:..."
59,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.960514,-82.506127,4.0,35,0,"{'BusinessParking': '{'garage': False, 'street...",", Pizza","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
93,RK6-cJ9hj53RzOlCBmpT-g,Impasto,,Tampa,FL,33611,27.890814,-82.502346,5.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",", Italian, Trucks,","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
140,zFvqulgAYOpSG2t1v8AZ-w,The Cake Drip,1625 W Snow Cir,Tampa,FL,33606,27.935753,-82.476470,4.0,40,1,"{'RestaurantsDelivery': 'False', 'RestaurantsT...","Desserts, Do-It-Yourself , Patisserie/Cake Shop","{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21..."
186,7Du9oW73YcYFmXdtU5aiSg,Kinjo's Japanese Restaurant,2875 Tyrone Blvd N,Saint Petersburg,FL,33710,27.797263,-82.733913,3.5,17,0,"{'RestaurantsPriceRange2': '2', 'RestaurantsGo...","Japanese, Sushi ,","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'..."
188,QjV4v7q_pt7tt3K1US7IHg,PDQ Temple Terrace,5112 E Fowler Ave,Tampa,FL,33617,28.054888,-82.399548,3.0,5,1,"{'RestaurantsReservations': 'False', 'Business...","Fast , Sandwiches, Chicken Shop,","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
224,Ucl9Vo5lwrUmYbV8Dv8X5g,O'Briens Irish Pub,15435 N Dale Mabry Hwy,Tampa,FL,33618,28.092360,-82.500588,4.0,108,1,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...",", Pubs, Nightlife, Arts & Entertainment, Irish...","{'Monday': '18:0-2:30', 'Tuesday': '15:0-1:0',..."
242,xbRbvXSAVZBEwaUUn3mzbg,Lee Roy Selmon's,14457 N Dale Mabry Hwy,Tampa,FL,33618,28.078735,-82.506310,3.5,42,0,"{'RestaurantsAttire': 'u'casual'', 'BusinessPa...","Salad, Burgers, Soul","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."


In [220]:
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords

# Define stop words
stop_words = set(stopwords.words('english'))

def get_most_frequent_category(category_string, global_counter):
    # Convert the category string to lowercase and split into individual words
    categories = category_string.lower().split(", ")
    
    # Remove empty strings and stop words
    categories = [word for word in categories if word and word not in stop_words]
    
    # If categories list is empty, return a default value (e.g., "unknown")
    if not categories:
        return "unknown"
    
    # Count the frequency of each word in the current category string
    local_counter = Counter(categories)
    
    # Find the most frequent word in the current category string
    most_frequent_word = max(local_counter, key=lambda word: global_counter[word])
    
    return most_frequent_word

# Assuming tampa_bay is your DataFrame
# Create a global frequency counter for all categories
global_counter = Counter()
for category_string in tampa_bay["categories"]:
    categories = category_string.lower().split(", ")
    categories = [word for word in categories if word and word not in stop_words and word.strip()]
    global_counter.update(categories)

# Apply the function to each row in the DataFrame
tampa_bay["final_category"] = tampa_bay["categories"].apply(lambda x: get_most_frequent_category(x, global_counter))

In [221]:
# count final_category values and show the 30 most common

tampa_bay["final_category"].value_counts().head(30)


final_category
nightlife                    931
sandwiches                   547
american (traditional)       310
coffee & tea                 271
pizza                        199
breakfast & brunch           198
american (new)               134
mexican                      134
sea                          133
burgers                      126
specialty                    109
desserts                     109
chinese                      102
japanese                      80
fast                          80
cafes                         67
italian                       57
beer                          55
event planning & services     55
latin american                55
grocery                       52
coffee & tea,                 52
ice cream & frozen yogurt     51
juice  & smoothies            49
nightlife,                    45
chinese,                      44
pizza,                        43
mexican,                      43
 trucks                       42
bakeries                    

Manual Cleaning of the categories

In [222]:

# removing all the commas and the spaces at the beginning and at the end of the string

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace(",", "", case=False, regex=True)
tampa_bay["final_category"] = tampa_bay["final_category"].str.strip()

# transforming sandwiches to "sandwiches and burgers"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("sandwiches", "sandwiches and burgers", case=False, regex=True)

#if "final_category" is exactly "burgers", transforming it to "sandwiches and burgers"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("burgers", "sandwiches and burgers", case=False, regex=True)
 
# removing "(traditional)" and "(new)",  

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("\(traditional\)", "", case=False, regex=True)
tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("\(new\)", "", case=False, regex=True)

# transforming "nightlife" to "pub and bars"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("nightlife", "pubs and bars", case=False, regex=True)
tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("pubs", "pubs and bars", case=False, regex=True)


# transforming "cafes" and "cafe" to "coffee and tea"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("cafes", "coffee & tea", case=False, regex=True)
tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("cafe", "coffee & tea", case=False, regex=True)

# if "breakfast" is present, transforming it to "bf & brunch"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("breakfast", "bf & brunch", case=False, regex=True)
tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("breakfast & brunch", "bf & brunch", case=False, regex=True)


# transforming "fast" to "fast food" and " trucks" to "food trucks"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("fast", "fast food", case=False, regex=True)
tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("trucks", "food trucks", case=False, regex=True)

# transforming "sea" to "seafood"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("sea", "seafood", case=False, regex=True)

# transforming "brunch & brunch" to "brunch", bars and bars to "bars", "sandwiches and sandwiches" to "sandwiches"

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("bars and bars", "bars", case=False, regex=True)

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("sandwiches and sandwiches", "sandwiches", case=False, regex=True)

tampa_bay["final_category"] = tampa_bay["final_category"].str.replace("brunch & brunch", "brunch", case=False, regex=True)

# removing rows with event planning & services and venues & event spaces in "final_category" column

tampa_bay = tampa_bay[~tampa_bay["final_category"].str.contains("event planning & services|venues & event spaces", case=False, na=False)]










In [225]:

tampa_bay["final_category"].value_counts().head(30)


final_category
pubs and bars                981
sandwiches and burgers       688
american                     483
coffee & tea                 395
pizza                        242
bf & brunch                  204
mexican                      177
seafood                      153
chinese                      146
specialty                    125
desserts                     112
fast food                     89
italian                       83
ice cream & frozen yogurt     83
japanese                      82
grocery                       78
latin american                59
juice  & smoothies            56
beer                          55
barbeque                      54
bakeries                      53
food trucks                   52
caribbean                     45
thai                          37
indian                        31
mediterranean                 30
cuban                         30
chicken wings                 30
salad                         28
sushi                       

KEEPING ONLY THE RESTAURANTS WITH 30 MOST COMMON CATEGORIES

In [228]:
# creating a new dataframe tampa_bay_30 with the 30 most common categories

tampa_bay_30 = tampa_bay[tampa_bay["final_category"].isin(tampa_bay["final_category"].value_counts().head(30).index)]

tampa_bay_30["final_category"].value_counts()


final_category
pubs and bars                981
sandwiches and burgers       688
american                     483
coffee & tea                 395
pizza                        242
bf & brunch                  204
mexican                      177
seafood                      153
chinese                      146
specialty                    125
desserts                     112
fast food                     89
italian                       83
ice cream & frozen yogurt     83
japanese                      82
grocery                       78
latin american                59
juice  & smoothies            56
beer                          55
barbeque                      54
bakeries                      53
food trucks                   52
caribbean                     45
thai                          37
indian                        31
mediterranean                 30
cuban                         30
chicken wings                 30
salad                         28
sushi                       

SAVING THE DATABASE AND ADAPTING REVIEW, CHECK-IN AND USERS

In [230]:
tampa_bay_30.to_csv(r"C:\Users\Giuseppe Branca\Desktop\Bootcamp miscellanea\PROYECTO\github\tampa_bay_30.csv", index=False)

In [232]:
tampa_bay_30.shape

(4708, 15)

FILTERING AND SAVING "REVIEWS"

In [233]:
# uploading reviews dataset to filter reviews of the 30 most common categories. creating chunks to avoid memory error

review = pd.read_json(folder + review_path, lines=True, chunksize=5000)


filtered_reviews = pd.DataFrame() # creating an empty df to store the filtered reviews

# iterating over the chunks

for chunk in review:
    # filtering the reviews for the businesses in tampa_bay_30
    chunk_filtered = chunk[chunk["business_id"].isin(tampa_bay_30["business_id"])]
    
    # append the filtered reviews to the final df
    filtered_reviews = pd.concat([filtered_reviews, chunk_filtered], ignore_index=True)

filtered_reviews.shape



(438563, 9)

In [234]:
filtered_reviews.head(10)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,OAhBYw8IQ6wlfw1owXWRWw,1C2lxzUo1Hyye4RFIXly3g,BVndHaLihEYbr76Z0CMEGw,5,0,0,0,"Great place for breakfast! I had the waffle, w...",2014-10-11 16:22:06
1,meGaFP7yxQdjyABrYDVeoQ,_jaJDV-qTBafatbObmtzpA,cg4JFJcCxRTTMmcg9O9KtA,1,0,1,0,Skip this train wreck if you are looking for d...,2018-02-11 03:11:20
2,PPgbLBvi34A6m7bKJfTwhw,3TL6HZ1JrKcNTvGDWKlrow,GyC36Pn0Q1-qHnqXys6yFg,1,0,0,0,Service and management terrible... After messi...,2013-12-07 13:17:13
3,OJO7m2zn3LAr011J1I7ppQ,JriXL8qqw_tJ1mpwtlBabg,1QVB0_-piu0GXes87BXeGw,5,1,0,0,Love this place...best hot dogs and chili dogs...,2017-10-11 01:27:15
4,LnKr0hwejzl71QmoQyTRDQ,7RU_xK1tEGlUvXfe0GvtEg,hAmuto6UndVroyd_DaD-TA,5,1,0,0,Not sure why it took until now for us to find ...,2018-03-04 00:20:24
5,UrGHZY5-pDs4uOP8SFPg-w,Nsm9V1s8JQyuYsShtiwplg,eUO4QA6nL2XdCeWuD7tkww,5,1,0,0,I stopped in for a drink yesterday and have to...,2017-05-26 10:53:47
6,wsFRDsHxz2mM_Ettgn1qQg,x8ErSBur0SsnL1lZwP5o4Q,qf4LecJDQWIt0gt6VJWFPw,4,0,0,0,We got diverted to Tampa and decided to make t...,2017-06-28 01:07:04
7,p3UM6VOJOGuY0iTxD-Topw,amIUTkKQlA4464-lJlwlaQ,oRZzKWdyMQld6N_y4UWkAg,4,0,0,0,Cinco de mayo means margaritas and sangria and...,2013-05-07 16:54:19
8,X-ibMbsz1nWAcetMERluBQ,6M7PMBWbBcp5VZ84AY3EFA,mLNSOU8Ki0Fm09xd6ZKkcA,2,1,0,0,I had the Chicken Marsala recommended by our w...,2015-01-18 19:04:28
9,kj5pjfid6pUP818rF0q6GQ,dWTrJdEPoZB8LkOPv_jxsA,cg4JFJcCxRTTMmcg9O9KtA,4,0,0,0,This was my first time dining at Ava and I enj...,2016-01-04 04:06:20


In [235]:
filtered_reviews.to_csv(r"C:\Users\Giuseppe Branca\Desktop\Bootcamp miscellanea\PROYECTO\github\filtered_reviews.csv", index=False)

FILTERING AND SAVING "USERS"

In [236]:
# uploading user dataset to filter users that made reviews of the 30 most common categories

user = pd.read_json(folder + user_path, lines=True, chunksize=5000)

filtered_users = pd.DataFrame() 

for chunk in user:
    chunk_filtered = chunk[chunk["user_id"].isin(filtered_reviews["user_id"])]
    filtered_users = pd.concat([filtered_users, chunk_filtered], ignore_index=True)

filtered_users.shape

(158911, 22)

In [237]:
filtered_users.to_csv(r"C:\Users\Giuseppe Branca\Desktop\Bootcamp miscellanea\PROYECTO\github\filtered_users.csv", index=False)

FILTERING AND SAVING CHECKIN

In [239]:
checkin = pd.read_json(folder + checkin_path, lines=True)

checkin.shape



(131930, 2)

In [240]:
filtered_checkin = checkin[checkin["business_id"].isin(tampa_bay_30["business_id"])]

filtered_checkin.shape

(4667, 2)

In [241]:
filtered_checkin.to_csv(r"C:\Users\Giuseppe Branca\Desktop\Bootcamp miscellanea\PROYECTO\github\filtered_checkin.csv", index=False)